In [ ]:
# stdlib
import os
from os import environ as env
from pathlib import Path
import sys

# syft absolute
import syft as sy
from syft.service.action.action_object import AnyActionObject
from syft.service.user.user_roles import ServiceRole
from syft.util.test_helpers.email_helpers import load_users
from syft.util.test_helpers.job_helpers import create_simple_query_job
from syft.util.test_helpers.job_helpers import create_wrong_syntax_query
from syft.util.test_helpers.job_helpers import load_jobs
from syft.util.util import find_base_dir_with_tox_ini
from syft.util.util import get_caller_file_path
from syft.util.util import is_interpreter_jupyter

# Prepare file

1. Start the cluster
- switch to pysyftclone, and go to 0.9.1 and 
    - export DEVSPACE_PROFILE=bigquery-scenario-tests
    - tox -e dev.k8s.destroy
    - tox -e dev.k8s.start
    - tox -e dev.k8s.deploy
2. Prepare the file
- go to normal pysyft dev (tox will create the right environment with 0.9.1 for you), edit some files

`tox -e migration.scenarios.k8s.prepare`

this will create migration.yaml

3. Start new cluster
- copy migration_k8s.yaml (in scenario notebooks), to grid/helm/examples/dev as migration.yaml
- export DEVSPACE_PROFILE=migrated-datasite
- tox -e dev.k8s.start
- tox -e dev.k8s.deploy

4. Run this File



In [ ]:
#! cp ./migration_k8s.yaml /Users/koen/workspace/PySyft/packages/grid/helm/examples/dev/migration.yaml

To prepare, run with the right profile

```
export DEVSPACE_PROFILE=bigquery-scenario-tests
tox -e dev.k8s.start
tox -e dev.k8s.deploy
```

In [ ]:
def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(base_dir, "test_helpers")
    sys.path.append(notebook_helper_path)

    notebook_helper_path = os.path.join(
        base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers"
    )
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()

In [ ]:
ROOT_EMAIL = "info@openmined.org"
ROOT_PASSWORD = "changethis"

In [ ]:
# when in k8s these are the default values
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [ ]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

In [ ]:
env["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"

# Connect email

In [ ]:
# third party
from email_helpers import get_email_server

In [ ]:
email_server, smtp_server = get_email_server()

# Login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

client = sy.login(url="http://localhost:8080", email=ROOT_EMAIL, password=ROOT_PASSWORD)

In [ ]:
# Check if this is a new server
migration_data = client.get_migration_data()

# assert len(migration_data.store_objects[User]) == 1
# assert UserCode not in migration_data.store_objects

In [ ]:
migration_data

# Load migration data

In [ ]:
migration_data_dir = Path(os.getenv("MIGRATION_DATA_DIR", "."))
blob_path = migration_data_dir / "migration_k8s.blob"
yaml_path = migration_data_dir / "migration_k8s.yaml"

print(f"Loading migration data from {str(blob_path.resolve())}")

In [ ]:
res = client.load_migration_data(blob_path)
assert isinstance(res, sy.SyftSuccess), res.message

In [ ]:
# from syft.service.migration.object_migration_state import MigrationData

# migration_data = MigrationData.from_file(blob_path)

# worker_pools = migration_data.get_items_by_canonical_name("WorkerPool")

# pool = worker_pools[0]

# images = migration_data.get_items_by_canonical_name("SyftWorkerImage")
# image_id = pool.image_id
# old_image = [img for img in images if img.id == image_id][0]

In [ ]:
# old_image

In [ ]:
# old_image.is_prebuilt

In [ ]:
# worker_pools[0]

In [ ]:
!docker build -f bigquery.dockerfile -t test .

In [ ]:
!docker image tag test k3d-registry.localhost:5800/openmined/test:dev-latest

In [ ]:
!docker push k3d-registry.localhost:5800/openmined/test:dev-latest

In [ ]:
new_config = sy.PrebuiltWorkerConfig(
    tag="k3d-registry.localhost:5800/openmined/test:dev-latest", description=""
)

print("submitting new prebuilt image...")
result = client.api.services.worker_image.submit(worker_config=new_config)

In [ ]:
image_uid = result.value.id  # or client.images[i].id

In [ ]:
result = client.api.services.worker_pool.launch(
    pool_name="bigquery-pool",
    image_uid=image_uid,
    num_workers=1,
)

In [ ]:
# sy.upgrade_custom_workerpools(client, blob_path)

# Emails

In [ ]:
client.register(
    name="abcdef",
    email="ab@de.org",
    password="abc",
    password_verify="abc",
    institution="comp",
    website="www.a.com",
)

In [ ]:
assert len(email_server.load_emails()["ab@de.org"]) == 1

# Post migration tests

In [ ]:
users = load_users(client, path="0.9.1_notebooks/users_k8s.json")
jobs = load_jobs(users, client, filepath="0.9.1_notebooks/jobs_k8s.json")

### Check users

In [ ]:
server_users = client.users

In [ ]:
server_users[0]

In [ ]:
server_user_names = [
    user.name for user in server_users if user.role == ServiceRole.DATA_SCIENTIST
]

In [ ]:
user_names = [user.name for user in users] + ["abcdef"]  # new registered user

In [ ]:
assert set(server_user_names) == set(user_names)

### Old jobs

In [ ]:
# submitted_jobs = [job for job in jobs if job.is_submitted]
reviewed_jobs = [job for job in jobs if job.admin_reviewed]
reviewed_jobs_should_succeed = [j for j in reviewed_jobs if j.should_succeed]
reviewed_jobs_should_fail = [j for j in reviewed_jobs if not j.should_succeed]

print(
    f"{len(reviewed_jobs)=}, {len(reviewed_jobs_should_succeed)=}, {len(reviewed_jobs_should_fail)=}"
)

In [ ]:
for job in reviewed_jobs_should_succeed:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method
    j = api_method(blocking=False)
    res = j.wait()

    if isinstance(res, sy.SyftError):
        raise sy.SyftException(public_message=res.message)

    result = res.get()
    job.result_as_expected = True

In [ ]:
for job in reviewed_jobs_should_fail:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method

    j = api_method(blocking=False)
    res = j.wait()
    if isinstance(res, sy.SyftError):
        job.result_as_expected = True
    else:
        raise sy.SyftException(public_message=f"failed, job didnt raise {type(j)}")

In [ ]:
expected_jobs = [job for job in jobs if job.result_as_expected]
print(f"got expected_jobs: {len(expected_jobs)} == reviewed_jobs: {len(reviewed_jobs)}")
assert len(reviewed_jobs) == len(expected_jobs)

### Use old DS to go through the flow again

In [ ]:
ds_client = users[0].client

In [ ]:
assert len(ds_client.api.services.api.api_endpoints()) == 3

In [ ]:
job = create_simple_query_job(users[0])

In [ ]:
response = ds_client.api.services.bigquery.submit_query(
    func_name=job.func_name, query=job.query
)
response

In [ ]:
assert isinstance(response, AnyActionObject)

In [ ]:
for request in client.requests:
    if request.code.service_func_name == job.func_name:
        request.approve()

In [ ]:
job_res = getattr(ds_client.code, job.func_name)(blocking=False)

In [ ]:
job_res.wait()

In [ ]:
# third party
from pandas import DataFrame

assert isinstance(job_res.result.get(), DataFrame)

In [ ]:
wrong_syntax_job = create_wrong_syntax_query(users[0])

In [ ]:
response = ds_client.api.services.bigquery.submit_query(
    func_name=wrong_syntax_job.func_name, query=wrong_syntax_job.query
)
response

In [ ]:
assert isinstance(response, AnyActionObject)

In [ ]:
for request in client.requests:
    if request.code.service_func_name == wrong_syntax_job.func_name:
        request.approve()

In [ ]:
job_res = getattr(ds_client.code, wrong_syntax_job.func_name)(blocking=False)

In [ ]:
assert isinstance(job_res.wait(), sy.SyftError)

In [ ]:
if server.server_type.value == "python":
    server.land()